# Introdução

(Muitos dos quais lerão esse texto, raramente em sua vida, se depararam com o problema de ter que utilizar outro
site de busca, além do Google, para encontrar algo que procura na internet. Porém, quando ela ainda estava em sua
fase “jovem”, isso era um problema comum.)
    
Praticamente todo mundo que você conhece usa o Google. Uns ou outros utilizam o Bing e algumas pessoas perdidas no
tempo usam o Yahoo!. Mas, algo razoável de se perguntar é: sempre foi assim? Provavelmente a resposta a essa
pergunta é: Nem sempre. [posso reler a introdução do livro para ter ideias boas do que adicionar aqui.] | Mas
então, **por que** o Google é _o Google_? A resposta disso em uma palavra é: PageRank.
  
    
    
    
## PageRank
    
PageRank é um algoritmo criado por Sergey Brin e Larry Page, os fundadores da Google, no final de década de 90, que
utiliza a estrutura de _links_ da Internet para dar uma classificação a suas páginas.

A ideia central do algoritmo é resumida na seguinte frase: uma página é importante se páginas importantes levam a
ela.  Em minha opinião, isso parece um pouco recursivo demais. Uma página será importante se outras páginas que
ligam a ela forem importantes. Agora, essas páginas são importantes porque outras páginas importantes ligam a elas.
Seguindo nesse raciocínio, chegamos na seguinte pergunta: quem são "as primeiras" páginas importantes? Essa
recursão, na verdade, é algo embutido no algoritmo, algo que é trabalhado para se calcular a classificação de cada
página. Conforme o texto, mais especificamente na seção de matemática, for progredindo, essa ideia ficará mais
clara e veremos que, na verdade, não precisam existir "as primeiras" páginas importantes.

Um fato curioso sobre o PageRank é que ele utiliza, praticamente, somente conceitos básicos da Álgebra Linear. Sim,
um dos grandes motivos do porquê a _Google_ ser tão poderosa no mercado tecnológico de hoje em dia é devido, no seu
início como empresa, à implementação de Álgebra Linear básica em algoritmos. Qualquer um que já fez algum curso de
Álgebra Linear possui as ferramentas básicas para entender _como_ e o _por que_ o PageRank funciona. Esse é justo o
foco central deste texto.


# A Matemática por trás

## Cadeias de Markov

Inicialmente, discutiremos a respeito de cadeias de Markov. Elas o ajudarão no entendimento e a ligar conceitos ao
longo do texto. Você provavelmente não conhece nada a respeito do que é uma cadeia de Markov. Portanto, aqui só
introduziremos seus conceitos fundamentais.

É verdade que, conforme comentado acima, apenas o conhecimento básico de Álgebra Linear já é suficiente para o
entendimento de como funciona o PageRank. Porém, conhecer os fundamentos de cadeias de Markov o ajuda a entender
uma área da matemática, provavelmente nova para você, leitor, e ter uma melhor noção "do que está acontecendo". E,
além do mais, não machuca ninguém.

Primeiramente, um _processo estocástico_ é um conjunto de variáveis aleatórias $\left\{X_t \right\}_{t=0}^{\infty}$
que possuem um conjunto de possíveis valores em comum $S = \left\{S_1, S_2, \dots, S_n\right\}$, que é chamado de
_espaço de estados_ para o processo. O parâmetro $t$ é geralmente pensado como tempo, e $X_t$ representa o estado
do processo no tempo $t$.  Por exemplo, considere o processo de navegar na Internet clicando em links que vão de
uma página a outra. O espaço de estados $S$ é o conjunto de todas a páginas da Internet e a variável aleatória $X_t$
é a página visitada no tempo $t$.

Uma _cadeia de Markov_ é um processo estocástico com a propriedade de ser um processo "sem memória". A
probabilidade da cadeia ir para algum estado depende somente do estado atual que ela se encontra e não dos
estados anteriores. Escrevendo isso em forma de equação temos,
$$
P\left(X_{t+1}=S_j | X_t=S_{i_t},X_{t-1}=S_{i_{i-1}},\dots,X_0=S_{i_0}\right) = P\left(X_{t+1}=S_j | X_t=S_{i_t}\right)
$$
para cada $t = 0, 1, 2, \dots$. A notação $P\left(E | F\right)$ representa a probabilidade condicional do evento
$E$ ocorrer dado que $F$ ocorreu. Por exemplo, é uma cadeia de Markov o processo de um usuário navegar na Internet
selecionando links que pertencem a página atual em que o usuário se encontra.

A _probabilidade de transição_ $p_{ij} = P\left(X_t=S_j | X_{t-1}=S_i\right)$ é a probabilidade de estar no estado
$S_j$ no tempo $t$ com o fato da cadeia ter passado no estado $S_i$ em $t-1$. Assim, é basicamente a probabilidade
de ir para $S_j$ de $S_i$ no tempo $t$.

A _matriz de probabilidade de transição_ é definida como sendo uma matriz em que seus elementos são probabilidades
de transição, $P_{n \times n}(t) = \left[p_{ij}(t)\right]$. Ela é não-negativa, visto que seus elementos são todos
probabilidades. Nota-se também que a soma dos elementos de cada linha deve ser 1, já que a soma das probabilidades
de ir para qualquer estado pertencente ao espaço de estados $S$ deve-se somar a 1. Matrizes que satisfazem essas
propriedades de serem não-negativas e que a soma dos elementos de cada linha ser igual a 1 são chamadas de
_matrizes estocásticas_. Portanto, dado um $t$ específico, $P(t)$ é uma matriz estocástica.  

Uma _cadeia de Markov_ é _estacionária_ se as probabilidades de transição não variarem com o tempo, isto é $p_{ij}(t) =
p_{ij}$ para todo $t$.

Um _vetor de distribuição de probabilidade_ (ou apenas "vetor de probabilidade") é definido com sendo um vetor $p^t
= (p_1, p_2, \dots, p_n)$ não-negativo de tal forma que a soma de seus elementos é igual a 1. 

Um _vetor de distribuição de probabilidade estacionário_ para uma cadeia de Markov estacionária que tem $P$ como matriz de
transição é um vetor $\pi$ de tal forma que $\pi^t = \pi^t P$.

Agora que conhecemos alguns novos conceitos de cadeias de Markov, vamos abordar o problema geral do PageRank.

## A Matriz Google

Para começarmos, suponha um conjunto $P$ com $n$ páginas da Internet dadas por $P_i \ (i = 1,2,\cdots,n)$. Suponha
também que as páginas desse conjunto possuem _links_ que vão para páginas do mesmo conjunto. Uma forma interessante
de visualizar as páginas de $P$ e as ligações entre elas é por meio de um grafo. Por motivos didáticos, usaremos um
conjunto $P$ com $n = 7$ páginas dadas pelo grafo abaixo.

![Grafo](Imagens/Grafo_1.png) 

Na imagem, os nós
(círculos) representam as páginas e as arestas (setas) representam as ligações entre as páginas.

Podemos representar esse grafo em um formato matricial. Seja $A$ uma matriz tal que $a_{ij}$ = 1, se a página $i$
possuí um link para a página $j$, e $a_{ij}$ = 0 caso contrário (a página $i$ não possui um _link_ para a página
$j$).  Acabamos de criar uma chamada matriz de _Adjacência_ do grafo. Assim, a matriz $A$ do grafo do conjunto de
páginas $P$ é dada por

$$ A = \begin{bmatrix} 
            0&1&0&0&0&0&0\\
            0&0&1&0&0&0&0\\
            1&0&0&1&0&0&1\\
            0&0&0&0&1&0&0\\
            0&0&0&0&0&1&0\\
            0&0&0&1&0&0&0\\
            0&0&0&0&0&0&0
\end{bmatrix} . $$

Agora, vamos olhar para a matriz $A$ de uma forma diferente. E se o elemento $a_{ij}$ representasse a probabilidade
de um usuário da Internet ir à página $j$ considerando o fato dele estar, agora no momento, na página $i$?
Observando $A$, vemos que essa interpretação nova não está consoante com a matriz e que um problema já visível está
em sua linha 3. Segundo nossa interpretação, se um usuário estiver na página 3, a probabilidade dele ir para página
1 é igual à 1. Porém, a chance dele ir para as páginas 4 e 7 também é 1, algo que não faz sentido. Um modo de
contornar esse problema é criar uma _nova_ matriz que tenha a mesma “cara” de  $A$ e que também, corresponda com
essa nova interpretação probabilística.

Um modo de criar essa nova matriz, digamos $H$, de forma que, as probabilidades sejam “justas” ou “sem viés” é pela
seguinte definição: o elemento $h_{ij}$ é igual à $(\sum_{k=1}^{n}h_{ik})^{-1}$ se a página $i$ possuí um _link_
para a página $j$ e $h_{ij} = 0$ caso contrário. Embora pareça um pouco complicado essa nova definição, saiba que a
única diferença entra ela e a de $A$ é que estamos “normalizando” as linhas não-nulas para que a soma entre seus
elementos seja igual à 1 e assim, faça sentido pensar nela como probabilidade. Assim, $H$ será dada por,

$$ H = \begin{bmatrix} 0&1&0&0&0&0&0 \\
            0&0&1&0&0&0&0\\
            1/3&0&0&1/3&0&0&1/3\\
            0&0&0&0&1&0&0\\
            0&0&0&0&0&1&0\\
            0&0&0&1&0&0&0\\
            0&0&0&0&0&0&0
\end{bmatrix} . $$

A matriz $H$ é uma chamada _matriz subestocástica_, visto que todas suas linhas não-nulas somam a 1. Porém, ainda
na matriz $H$ temos um problema. E a linha 7? Ela, por sua vez, possuí uma linha completa de zeros, o que quer
dizer pela nossa interpretação que, se um usuário estiver na página 7, a probabilidade dele ir para qualquer outra
página (de $P$) é zero. O que intuitivamente quer dizer que ele ficará na página 7 **para sempre**.  Obviamente,
isso é algo que não queremos que aconteça com nosso modelo, que a página 7 seja um “buraco negro” para nossos
usuários, em que, se eles chegarem lá, viverão para sempre.

Faremos o seguinte para contornar esse fato: se uma linha contiver apenas zeros, ela será alterada de forma que,
todos seus elementos sejam iguais à $\frac{1}{n}$, em que $n$ é o número de páginas de $P$ (dimensão de $H$). O que
isso interpretativamente faz é que caso um usuário chegue a uma página que não possua ligação alguma com outra, ele
se direcionará a alguma página de $P$. É como se ele escolhesse uma página do seu histórico de navegação
“aleatoriamente”.

Assim, a matriz $H$ “atualizada”, o qual chamaremos de $S$, é dada por

$$ S = \begin{bmatrix} 0&1&0&0&0&0&0 \\
            0&0&1&0&0&0&0\\
            1/3&0&0&1/3&0&0&1/3\\
            0&0&0&0&1&0&0\\
            0&0&0&0&0&1&0\\
            0&0&0&1&0&0&0\\
            1/7&1/7&1/7&1/7&1/7&1/7&1/7 
\end{bmatrix} . $$

Recorde que cada elemento de $S$, $s_{ij}$, representa a probabilidade de um usuário ir de uma página $i$ para uma
página $j$, assim $s_{ij} = P\left(X_{t+1} = S_j | X_t = S_i\right)$, com $X_t$ sendo a página visitada no tempo
$t$, $S_j$ e $S_i$ sendo, respectivamente, a página $j$ e a página $i$. Portanto, devido a nossa construção, $S$ é
uma _matriz de probabilidade de transição estocástica_ com cada elemento seu $s_{ij}$ sendo uma _probabilidade de
transição_ entre páginas.

Porém, _incrivelmente_, ainda há mais um problema (e o último) com nosso modelo. Se, você leitor, observasse a
matriz $S$ por um tempo suficientemente grande, provavelmente iria notar a seguinte propriedade da matriz: Se um
usuário estiver na página 4, ele irá para a página 5. Se estiver na página 5, ele irá para a página 6. E se estiver
na página 6, ele irá para a página 4. E assim por diante, para sempre. Criando assim um _loop_ eterno da navegação
do mesmo. E com isso, a partir do momento em que entra pela primeira vez em uma dessas páginas, as outras ($P_i$
com $i= 1,2,3,7$) efetivamente “não existirão” mais em nosso modelo, dessa maneira, não será possível quantificar
algum tipo de classificação numérica para as mesmas, mas apenas para aquelas que estão no ciclo.

Para que o fato discutido acima não ocorra, consideraremos mais uma, e última modificação no comportamento do nosso
usuário. Agora, antes de simplesmente selecionar um _link_ na página que atualmente se encontra, o usuário terá uma
probabilidade $1 - \alpha$ (com $\alpha \in (0,1)$) de ir para uma página qualquer de $P$. Isso, além trazer
propriedades que garantirão o êxito de nosso modelo, algo que veremos posteriormente, ela também traz a ele um
comportamento esperado de qualquer um que navega a Internet. É razoável esperar de uma pessoa que ela não somente
vá sendo levada site a site seguindo apenas os _links_ da página em que se encontra. Ela também pode, por exemplo,
entrar em algum site em que a aba está aberta em seu navegador, ou também, abrir um de seu histórico por livre e
espontânea vontade.

Seja o vetor $e \in \mathbb{R}^n$ um vetor coluna com todas entradas iguais a 1 
$$ e = 
\begin{bmatrix} 
                1 \\
                1 \\
                \vdots \\
                1
\end{bmatrix}
. 
$$
A nova matriz criada a partir de $S$ será a chamada _matriz Google_ $G$ que é dada pela seguinte equação:

$$ G = \alpha S + (1 - \alpha)1/nee^T . $$

Em que $1/nee^T \in \mathbb{R}^{n \times n}$ é uma matriz de “teleportação aleatória”, o qual todos seus elementos
são iguais à $\frac{1}{n}$. Em nosso exemplo, escolhendo $\alpha = 0.85$, a matriz G é

$$ G = 0.85 \begin{bmatrix} 0&1&0&0&0&0&0 \\
            0&0&1&0&0&0&0\\
            1/3&0&0&1/3&0&0&1/3\\
            0&0&0&0&1&0&0\\
            0&0&0&0&0&1&0\\
            0&0&0&1&0&0&0\\
            1/7&1/7&1/7&1/7&1/7&1/7&1/7 
\end{bmatrix}
+ 0.15 \begin{bmatrix} 1/7&1/7&1/7&1/7&1/7&1/7&1/7\\
            1/7&1/7&1/7&1/7&1/7&1/7&1/7\\
            1/7&1/7&1/7&1/7&1/7&1/7&1/7\\
            1/7&1/7&1/7&1/7&1/7&1/7&1/7\\
            1/7&1/7&1/7&1/7&1/7&1/7&1/7\\
            1/7&1/7&1/7&1/7&1/7&1/7&1/7\\
            1/7&1/7&1/7&1/7&1/7&1/7&1/7
\end{bmatrix} . $$

Note que, que nem a matriz $S$, $G$ também é uma _matriz de probabilidade de transição estocástica_ com cada
elemento seu $g_{ij}$ sendo uma _probabilidade de transição_ entre páginas.

## O Vetor do PageRank

Seguindo a filosofia de que _uma página é importante se páginas importantes direcionam a ela_, vamos definir uma
fórmula para calcular o $PageRank$ de uma página. Uma possível fórmula poderia simplesmente ser a soma dos
PageRanks das outras páginas que apontam para ela. Assim sendo, $$ r(P_i) = \sum_{P_j \in B_{P_i}}{r(P_j)}, $$ em
que $r(P_i)$ é o PageRank da página $i$, $r(P_j)$ o da página $j$ e $B_{P_i}$ é o conjunto das páginas que apontam
para a página $i$. Como cada $r(P_i)$ é um _rank_, temos que $r(P_i) > 0$. Para que o valor de algum $r(P_i)$ não
"exploda", será também imposto a condição de que $\sum_{i=1}^n r(P_i) = 1$.

Porém, se questione do seguinte: imagine duas páginas os quais possuem o mesmo PageRank. Uma dessas páginas possui
apenas um link para uma página qualquer, enquanto a outra possui links para cem páginas. O peso do link da primeira
página deve ser o mesmo que o peso de cada link da segunda?

Para os criadores do Google, a resposta é não. Quanto mais links uma página possui, a "importância" de cada um
desses links deve valer menos. Portanto, para a fórmula do PageRank, é preciso ter um fator de peso que mede o quão
"expressivo" é um link.

Assim, a fórmula inicialmente usada por Sergey Brin e Larry Page é 
$$
r(P_i) = \sum_{P_j \in B_{P_i}}\frac{r(P_j)}{|P_j|}, 
$$
em que $|P_j|$ é o número de links de $P_j$.

O problema óbvio com essa fórmula é que simplesmente não sabemos os valores de $r(P_j)$. A forma utilizada para
lidar com isso será aplicar a fórmula sucessivamente para as páginas de $P$, utilizando, a cada nova iteração, os
valores obtidos para $r(P_j)$ da iteração prévia e torcer para que os valores de $r(P_j)$ convirjam para algo
_estável_.  Assim, introduzindo uma nova notação, temos

$$ r_{k+1}(P_i) = \sum_{P_j \in B_{P_i}}\frac{r_k(P_j)}{|P_j|}. $$

Para que a fórmula acima faça sentido, é preciso ainda saber quem são os $r_0(P_i)$, $i = 1,\dots, n$, que é justo
nossa dúvida inicial discutida na seção "PageRank". Como inicialmente conhecemos nada sobre as páginas $P_i$, é
razoável pensar que todas, no começo do processo, valem o mesmo. Portanto a condição inicial dos $r_k(P_i)$ será
$r_0(P_i) = \frac{1}{n}$ para $i = 1,\dots, n$.

Agora que a mágica começa a acontecer. Pela equação acima, temos um sistema de $n$ equações em que cada uma depende
da outra. Em todas, os $r_k(P_i)$ são "pesados" por diferentes coeficientes em cada equação, contando também com a
"pesagem" por $0$. E ainda por cima, há um somatório em todas equações. Isso lembra uma multiplicação matriz-vetor.
E de fato, esse sistema realmente pode ser representado desta forma.

Os $r_k(P_i)$, $i = 1, \dots, n$ serão as componentes de um vetor $\pi^k$ e os "pesos" $|P_i|$, $i = 1,\dots, n$
farão parte da matriz de coeficientes.

Visto que há condições impostas aos todos $r_k(P_i)$ $i=1,\dots,n$, $\pi^k$ também as terá. Primeiramete, temos que
$r_k(P_i) = \pi_i^k > 0$ o que implica em $\pi^k > 0$. E também que como $\sum_{i=1}^n r_k(P_i) = \sum_{i=1}^n
|r_k(P_i)| = 1$ isso significa que $\|\pi^k\|_1 = 1$. Note que $\pi^k$ é um _vetor de distribuição de probabilidade_.

Agora você leitor, consegue lembrar de alguma matriz que era composta por "pesos", que juntos somavam a $1$ e que
dependiam somente do quanto de _links_ uma página $P_i$ possuía para outras páginas? Essa é justo a matriz $H$.
Para lembrança, a matriz $H$ do exemplo dado na seção "A Matriz Google" é

$$ 
H = 
\begin{bmatrix}
            0&1&0&0&0&0&0 \\
            0&0&1&0&0&0&0\\
            1/3&0&0&1/3&0&0&1/3\\
            0&0&0&0&1&0&0\\
            0&0&0&0&0&1&0\\
            0&0&0&1&0&0&0\\
            0&0&0&0&0&0&0
\end{bmatrix}
. 
$$

O que nos falta agora é se decidir se a multiplicação será dada pela direita, $\pi^{k+1} = H\pi^k$, ou pela
esquerda, $(\pi^t)^{k+1} = (\pi^t)^kH$. Como os elementos de uma coluna $i$ de $H$ representam as probabilidades do
usuário na página da linha $j$ ir a página $i$, esses são justos os "pesos" da fórmula. Assim, a multiplicação
matriz-vetor deve ser feito pela esquerda.

$$
(\pi^t)^{k+1}
=
(\pi^t)^kH . 
$$

Porém, como já foi discutido na seção "A Matriz Google", sabemos que a utilização da matriz $H$ trás problemas para o
modelo. Portanto, em vez de $\pi^k$ ser multiplicado por $H$, ele será multiplicado por $G$. Desse modo tem-se,

$$
(\pi^t)^{k+1}
=
(\pi^t)^kG . 
$$

Uma implicação "ruim" do uso da $G$ na equação é que ela não nos dará o "real" ranqueamento de cada página. Isso se
deve ao fato de $G$ ser o resultado de uma soma entre $S$ e $ee^t$. A matriz $S$ preserva a estrutura de links das
páginas da Internet e lida de forma "especial" com páginas sem links. Enquanto $ee^t$, que é uma matriz de
teleportação aleatória, não preserva a estrutura. Ainda assim é necessário a utilização de $G$ para, pelo menos,
obter uma aproximação do _rank_ "real" das páginas. Não é possível de se calcular este _rank_ "real", utilizando
somente $S$. Apenas o uso dela pode acarretar no fenômeno do usuário fictício seguir um caminho periódico
previsível entre as páginas, algo visto na seção anterior. Ou, de  forma mais geral, o usuário pode navegar somente
num conglomerado de páginas sem ter a possibilidade do mesmo conseguir  "chegar" em alguma página fora desse
conglomerado. Em qualquer um desses casos, o _PageRank_ das páginas que não pertencem a esse conglomerado iria
tender a zero conforme o cálculo dos _ranks_ for sendo executado. O que implica na falha do modelo para o cálculo
do _rank_ dessas páginas, algo não desejado.

Esperamos que os elementos de $\pi^k$, $\pi_i^k = r_k(P_i)$, convirjam para um valor estável depois de várias
iterações, isso significa que para um $k$ "grande", $\pi^k \approx \pi$, para certo $\pi$. Assim podemos escrever
que para $k \rightarrow \infty$, $\pi^k = \pi$ em que $\pi$ que é chamado de o _vetor do PageRank_ satisfaz 
$$
\pi^t
=
\pi^tG. 
$$
 
Deste modo, o problema do _PageRank_ se reduz a encontrar o _vetor probabilidade_ $\pi$ que satisfaz as condições $\pi > 0$,
$\|\pi\|_1$ = 1 e também $\pi^t = \pi^tG$.

[Aqui seria bom discutir o fato de não sabermos se o vetor $\pi$ existe, se ele é único e também se o método
proposto para calculá-lo "funciona", se ele converge ou não.]

## Autovalor e Autovetor

Agora antes de continuar, vamos fazer uma pequena pausa no racicíonio para relembrar o conceito de autovalor e
autovetor que será importante para o entendimento deste texto.

Você provavelmente está acostumado com a seguinte definição de autovalor e autovetor: dada uma matriz qualquer $A
\in \mathbb{R}^{n \times n}$ e um vetor $v \in \mathbb{R}^n$, com $v \neq 0$, $v$ é autovetor de $A$ associado ao
autovalor $\lambda$ se $$ Av = \lambda v,$$ para algum $\lambda \in \mathbb{R}$. Uma forma de se calcular os
autovalores $\lambda$ é pelo _polinômio característico_ de $A$, $p(A) = det(A-\lambda I)$.

Porém, mesmo A possuindo apenas entradas reais, as raízes de $p(A)$ podem assumir valores complexos. Assim, se nos
restringirmos a valores reais para os autovalores $\lambda$, é possível que haja a "perda" de certos $\lambda$.
Isso implicaria que a "quantidade", multiplicidade algébrica, de autovalores de $A$ seja menor do que $n$, sua
dimensão, o que implicaria em problemas para nossa análise. Portanto, para que nenhum $\lambda$ fique de fora, será
adotada a seguinte definição de autovalor e autovetor:

Dada uma matriz $A \in \mathbb{C}^{n \times n}$ e um vetor $v \in \mathbb{C}^n$, com $v \neq 0$, $v$ é autovetor de
$A$ associado ao autovalor $\lambda$ se $$ Av = \lambda v,$$ para algum $\lambda \in \mathbb{C}$.

O conjunto de autovalores $\lambda$ que uma matriz $A$ possui será chamado de _espectro_ de A, denotado por
$\sigma(A)$.  O valor absoluto dos maiores $\lambda$ em módulo, $|\lambda _i| \geq |\lambda _j|$, será chamado de
_raio espectral_, denotado por $\rho(A)$. 

Uma observação importante é que existem tanto autovetor à direita, $Ax = \lambda x$, quanto à esquerda, $y^tA =
\lambda y^t$. Os autovetores à direita são os que estamos mais habituados a lidar. Mas não se preocupe, se você
entende autovetores à direita você  também entende o à esquerda. É praticamente a mesma ideia. Algo não difícil de
provar é que o espectro $\sigma(A)$ de uma matriz A, o conjunto dos autovalores de uma matriz, dos autovetores à
direita e dos à esquerda de $A$ é o mesmo (tente provar você mesmo!), assim, se existem autovetores à direita para
um certo $\lambda$ irá também existir autovetores à esquerda para o mesmo $\lambda$. Porém há sim, certas
diferenças entre ambos, como por exemplo que se um vetor $v$ é autovetor à direita associado a um autovalor
$\lambda$ isso não implica que $v^t$ também será autovetor à esquerda associado a $\lambda$.

Dito isso e olhando a equação que chegamos na seção anterior, $\pi^t = \pi^tG$, sabemos agora que o problema do
_PageRank_ é também um problema de autovalor e autovetor. No caso, queremos encontrar o autovetor à esquerda de
$G$, que satisfaz certas condições, associado ao autovalor 1.

Como tanto $\pi$, $\lambda = 1$ e $G$ pertencem aos reais, temos que todas as iterações $\pi^k$ pertencerão aos
reais.  Assim, mesmo aparecendo números complexos na nossa nova definição de autovalor e autovetor, só utilizaremos
números reais para chegar ao _vetor do PageRank_ $\pi$. Portanto não precisamos nos preocupar com a aritmética
complexa no problema do _PageRank_.

Agora que abordamos o conceito de autovalores e autovetores, podemos começar a encontrar respostas para os
questionamentos a respeito do _vetor de probabilidade_ $\pi$. Primeiramente, para discutir sobre a existência e unicidade,
utilizaremos um teorema importante na área de Álgebra Linear, o chamado teorema de Perron.

## Perron

O teorema de Perron infere propriedades de um autovalor específico de uma matriz $A$ que satisfaz apenas duas
condições: A é quadrada, $A \in \mathbb{R}^{n \times n}$, e que todos seus elementos sejam positivos, $a_{ij} > 0,
i,j = 1,2,\dots,n$.

Dentre todas as afirmações que o teorema diz, as mais importantes para nós são as seguintes: Seja A uma matriz
quadrada com apenas entradas positivas. Existe um único autovetor $p$ à direita de $A$, com $p > 0$ e $\|p\|_1 =
1$, chamado _vetor de Perron_, associado a um autovalor $\lambda = r > 0$, chamado de _raiz de Perron_, com
multiplicidade algébrica igual à 1. Além de $p$ ser único, ele ainda possui a "unicidade" de, exceto de seus
múltiplos positivos, não haver outros autovetores positivos à direita de $A$, independentemente do autovalor. E,
além de tudo isso, $r$ é o maior autovalor em magnitude de $A$, com $|r| = \rho(A)$. 

No teorema está sendo apenas descrito o _autovetor de Perron_ à direita tal que $Ap = rp$. Porém ainda assim existe
o _autovetor de Perron_ à esquerda, digamos $q$, com as mesmas condições e propriedades de $p$ associado também a
_raiz de Perron_ $r$ em que $q^tA = rq^t$.


A _matriz Google_ $G$ satisfaz as duas condições iniciais do teorema, assim deve existir um autovalor $\lambda = r$
de $G$ que possui as propriedades descritas acima. O que nos resta agora é encontrá-lo.

Note que a multiplicação entre $G$ e $e$, o vetor coluna com todas entradas iguais a 1, representa a soma dos
elementos das linhas de $G$, que por construção é igual à 1. Deste modo temos que,  
$$ Ge = e. $$

Esse resultado implica que o vetor $e$ é um autovetor à direita de $G$ com autovalor $\lambda = 1$. Sabemos pelo
Teorema de Perron que autovetores positivos devem ter como autovalor associado $\lambda = r$. Como $e> 0$ e tem
como autovalor associado $\lambda = 1$, concluímos que $r = 1$.

Agora que sabemos que o _autovalor de Perron_ de $G$ é 1, podemos nos apropriar das propriedades do teorema. Como
o _vetor do PageRank_ $\pi$ satisfaz
$$ \pi^t = \pi^tG,\; \pi > 0 \; e \; \|\pi\|_1 = 1,$$
temos que $\pi$ é o _vetor de Perron_ à esquerda de $G$. Assim, utilizando $G$ para se calcular o vetor $\pi$, sua
existência e unicidade são garantidos graças ao Teorema de Perron.

Como agora sabemos que o _vetor de probabilidade_ $\pi$ existe e é único, só nos resta ainda um questionamento: é
possível calcular $\pi$ pelo método de iterações proposto?

## Método da Potência

O método de iterações proposto na seção Fórmula para o PageRank é um caso específico do Método da Potência. O
Método da Potência é uma técnica iterativa usada para determinar o autovalor dominanante de uma matriz, isto é, o
maior autovalor em magnitude. O método, além de calcular um autovalor, obtém também um autovetor
associado.

Para poder aplicar o Método da Potência, devemos ter uma matriz $A_{n \times n}$ diagonalizável (possui $n$
autovetores linearmente independentes) com um maior autovalor em magnitude $\rho(A) = |\lambda_1|$, de tal forma que
$$ 
|\lambda_1| > |\lambda_2| \geq |\lambda_3| \geq \dots \geq |\lambda_n|.  
$$
Observe que o fato de $|\lambda_1| > |\lambda_i|$ para $i = 2, \dots, n$ impõe que $\lambda_1 \in \mathbb{R}$. Caso
a parte imaginária de $\lambda_1$ fosse diferente de zero, existiria um autovalor conjugado $\bar{\lambda}_1 \neq
\lambda_1$ de tal forma que $|\bar{\lambda}_1| = |\lambda_1|$, que quebraria a hipótese.

Seja $\{V^{(1)}, V^{(2)}, V^{(3)}, \dots, V^{(n)}\}$ um conjunto de $n$ autovetores linearmente independentes
associados aos seus respectivos autovalores $\lambda_i$. Podemos representar um vetor $x$ qualquer utilizando os
autovetores $V^{(i)}$'s como base. Assim,
$$
x = \sum_{i = 1}^n \alpha_i V^{(i)}.
$$
Aplicando a matriz $A$ a $x$ obtemos,
$$
Ax = \sum_{i = 1}^n \alpha_i AV^{(i)} = \sum_{i = 1}^n \lambda_i \alpha_i V^{(i)}.
$$
Aplicando a matriz $A$ a $Ax$ obtemos,
$$
AAx = A^2x = \sum_{i = 1}^n \lambda_i \alpha_i AV^{(i)} = \sum_{i = 1}^n \lambda_i^2 \alpha_i V^{(i)}.
$$
Fazendo esse mesmo processo $k$ vezes chegamos em,
$$
AA^{k-1}x = A^kx = \sum_{i = 1}^n \lambda_i^{k-1} \alpha_i AV^{(i)} = \sum_{i = 1}^n \lambda_i^k \alpha_i V^{(i)}.
$$
Fatorando $\lambda_1^k$ do lado direito da última equação vemos que,
$$
A^kx = \lambda_1^k \sum_{i = 1}^n \left(\frac{\lambda_i}{\lambda_1}\right)^k \alpha_i V^{(i)},
$$
e assim, chegamos em,
$$
A^kx = \lambda_1^k \left(\alpha_1 V^{(1)} + \left(\frac{\lambda_2}{\lambda_1}\right)^k
\alpha_2 V^{(2)} + \dots + \left(\frac{\lambda_n}{\lambda_1}\right)^k \alpha_n V^{(n)} \right).
$$
Observe que como $|\lambda_1| > |\lambda_i|$ para $i = 2,\dots,n$, temos que $(\lambda_i/\lambda_1) < 1$ e assim,
$$
\lim_{k \to \infty} \left(\frac{\lambda_i}{\lambda_1}\right)^k = 0, \; \; \text{para } i = 2, \dots, n.
$$
Portanto conforme aumenta o número de iterações $k$, os termos de $\lambda_1^k \sum_{i=1}^{n}
\left(\frac{\lambda_i}{\lambda_1}\right)^k \alpha_i V^{(i)}$ vão tendendo cada vez mais a $\lambda_1^k \alpha_1 V^{(1)}$. Para que
possamos tomar o limite de $k$ tendendo ao infinito, devemos tomar o cuidado para que $\lambda_1^k$ convirja. Para
isso, só nos resta a opção $\lambda_1 = 1$. Caso $|\lambda_1|$ fosse menor do que 1, teríamos $\lim_{k \to \infty}
\lambda_1^k = 0$, que não nos interessa. Caso $|\lambda_1| > 1$, o limite $\lim_{k \to \infty}\lambda_1^k$ iria
divergir para o infinito. E se caso $\lambda_1 = -1$, $\lambda_1^k$ não convergiria para um valor fixo conforme
$k$ aumenta. Assim, assumindo $\lambda_1 = 1$ e tomando o limite de $k$ tendendo ao infinito, vemos que,
$$
\lim_{k \to \infty}A^kx =  \lim_{k \to \infty} \lambda_1^k \left(\alpha_1 V^{(1)} + \left(\frac{\lambda_2}{\lambda_1}\right)^k
\alpha_2 V^{(2)} + \dots + \left(\frac{\lambda_n}{\lambda_1}\right)^k \alpha_n V^{(n)} \right) = \lim_{k \to \infty} 
\lambda_1^k \alpha_1 V^{(1)}
$$
e assim,
$$
\lim_{k \to \infty}A^kx = \alpha_1 V^{(1)}.
$$

Portanto, chegamos na conclusão que, pela equação acima, $A^kx$ tende a um par de autovalor-autovetor, com
autovalor $\lambda = 1$ e autovetor associado $V = \alpha_1 V^{(1)}$ sempre que $A$ for diagonalizável e tiver
$\lambda_1 = 1$ como o maior autovalor em módulo. A única forma do limite dar um resultado ``insatisfatório", é
caso $\alpha_1 = 0$, isto é, a contribuição de $V^{(1)}$ na representação de $x$ pela base $\left\{V^{(1)},
V^{(2)}, \dots, V^{(n)}\right\}$ for zero. Contudo, se isso acontecesse, o que é raro, poderíamos apenas usar
algum outro $x_0$ que tenha $\alpha_1 \neq 0$ em $A^kx_0$.  

Desenvolvemos até aqui o Método da Potência para $\lambda_1 = 1$, que é o caso que justamente nos interessa. No
caso que $\lambda_1 \neq 1$, o Método da Potência é levemente diferente. Nesse, $A^kx$ é normalizada a cada
iteração, com o intuito de não permitir que tende a zero ou divirja.  Todavia, a ideia por trás e o resultado final
obtido será análogo ao que foi desenvolvido.

Aplicando o resultado acima para a matriz $G$ e supondo que ela seja diagonalizável, devemos analisar os maiores
autovalores em magnitude de $G$ para se certificar que o Método da Potência aplicada a ela converge. Já sabemos da
seção Perron que $\lambda_1 = 1 = \rho(G)$, que implica que $1 \geq |\lambda_2| \geq \dots \geq |\lambda_n|$.
Porém, deve-se ter $\lambda_1 = 1$ como o único autovalor no raio espectral, isto é, $1 > |\lambda_i|$ para $i =
2,3, \dots, n$. É possível provar, no entanto, não o faremos aqui, que o parâmetro $\alpha \in (0,1)$ da equação
que define $G$, 
$$
G = \alpha S + (1 - \alpha) ee^t, 
$$
é o segundo maior autovalor em magnitude de $G$. Como $\lambda_2 = \alpha < 1$, temos que $1 > \alpha \geq
|\lambda_3| \geq \dots \geq |\lambda_n|$.

Como $G$ satisfaz todas as hipóteses para a convergência do Método da Potência, temos que o limite $\lim_{k \to
\infty} G^k x$, converge para um autovetor $V$ associado ao autovalor $\lambda = 1$, assim
$$
\lim_{k \to \infty} G^kx = V
$$

Como $\pi$ e $V$ são autovetores associados de $\lambda = 1$ que, devido ao Teorema de Perron, possui
multiplicidade algébrica igual a 1, sabemos que $V$ é um múltiplo de $\pi$. Devido ao fato de $\|\pi\|_1 = 1$, 
$\pi > 0$ e $V$ ser $V > 0$ ou $V < 0$, temos que

$$
\pi = \text{sgn}(V) \frac{V}{\|V\|_1},
$$

em que $\text{sgn}(V)$ = 1 caso $V > 0$ e $\text{sgn}(V)$ = -1 caso $V < 0$.

Assim, chegamos na conclusão que o _vetor do PageRank_ $\pi$ é calculável pelo Método da Potência aplicado a $G$.


[Cadeia de Markov $\rightarrow \dots \rightarrow$ Método da Potência $\rightarrow \dots$]

[Método da Potência: Supor G diagonalizável ?]

[dúvida: já chamei $\pi$ de vetor do PageRank ?]